In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import re
from scipy import stats

sns_red = '#e77c8d'
sns_blue = '#5ea5c5'
sns_green = '#56ad74'
sns_purple = '#a291e1'

%matplotlib inline

DATA_PATH = './../data'

# 1. Opening the files

In [ ]:
pattern = re.compile(r'^pc.*\.xls$', re.IGNORECASE)
files = [f for f in glob('*', root_dir=DATA_PATH) if pattern.match(f)]

if files:
    print('Found Files:')
    for i, file in enumerate(files):
        print(f'{i+1} - "{file}"')

    # this is used when saving files
    common_name = re.findall(r'pc.*\(\d{1,3}mgml\)', files[0], re.IGNORECASE)[0].replace(' ', '_').lower()
    print(f"\nFiles related to -> {common_name}")
    if not common_name:
        common_name = ''

else:
    raise Exception('No files were found !')

In [ ]:
interested_in = ['Storage modulus', 'Loss modulus', 'Tan(delta)', 'Temperature']

dfs = [pd.read_excel(f"{DATA_PATH}/{file}",
                     index_col=None,
                     header=1,
                     sheet_name=1,
                     usecols=interested_in,
                     na_values=("NA", "N/A", "na", "n/a", "NULL", "null", "Not documented", "Not Documented", 'nan', '')).drop(labels=0, axis=0).reset_index(drop=True)
       for file in files]

Another way (more robust) to get the same result, but it takes longer to run
```python
interested_in = ['Storage modulus', 'Loss modulus', 'Tan(delta)', 'Temperature']
dfs = []

for file in files:
    t = pd.read_excel(file,
                      index_col=None,
                      header=[1,2],
                      sheet_name=1,
                      na_values=("NA", "N/A", "na", "n/a", "NULL", "null", "Not documented", "Not Documented", 'nan', ''))
    t.columns = t.columns.get_level_values(0)
    t = t.loc[:, interested_in]
    dfs.append(t)
```

# 2. Calculating the average values
Just averaging out the data will account for the random uncertainty in the measurements, but not the systematic uncertainty. To account for the systematic uncertainty, we need to calculate the standard deviation of the data and use it to calculate the error bars.

The uncertainty in the average value is given by the standard deviation of the data divided by the square root of the number of measurements.
(video -> (Max - Min) / 2 )

In [ ]:
# TODO: use temp bins to create the average values rather than just going row by row, maybe you can use pd.cut or pd.categorical see below
# minv = int(np.min(temp))
# maxv = ceil(np.max(temp)) + 10
# bins = list(range(minv, maxv, round((maxv - minv) / 5))) + [maxv + 1]
# temp_df['new_col'] = pd.cut(df.loc[:, col].copy(), bins)

min_length = min({df.shape[0] for df in dfs})
max_length = max({df.shape[0] for df in dfs})

# checking the Excel files length is acceptable
# change the "5" to any number you think is acceptable, or change the 'ignore_allowable_diff' to True if you want to ignore this part
allowable_diff = 5
ignore_allowable_diff = False

if max_length - min_length > allowable_diff and not ignore_allowable_diff:
    message = ''
    for i, file in enumerate(files):
        message = message + f'{file} -> {len(dfs[i])} rows\n'
    raise Exception(f"The Excel files are more than {allowable_diff} rows different in length \n{message}")

av_df = pd.DataFrame(columns=interested_in)
num_of_files = len(dfs)

# TODO: It is not recommended to build DataFrames by adding single rows in a for loop. Build a list of rows and make a DataFrame in a single concat.

for i in range(min_length):
    for col in interested_in:
        av_df.loc[i, col] = round(sum([df.loc[i, col] for df in dfs]) / num_of_files, 5)

av_df.head(3)

# Exporting the averaged data
av_df.to_excel(f'{DATA_PATH}/av_data_{common_name}.xlsx', index=None)

# 3. Looking at the data

In [ ]:
new_df = av_df.copy()
x = new_df.loc[:, 'Temperature'].to_numpy().astype(np.float16)
y1 = new_df.loc[:, 'Storage modulus'].to_numpy().astype(np.float16)
y2 = new_df.loc[:, 'Loss modulus'].to_numpy().astype(np.float16)
y3 = new_df.loc[:, 'Tan(delta)'].to_numpy().astype(np.float16)

# Plotting
fig, ax = plt.subplots(dpi=150)
ax.set_yscale('log')

for df in dfs:
    sns.scatterplot(data=df, x='Temperature', y="Storage modulus", ax=ax, s=10, color=sns_red)
    sns.scatterplot(data=df, x='Temperature', y="Loss modulus", ax=ax, s=10, color=sns_blue)
    sns.scatterplot(data=df, x='Temperature', y="Tan(delta)", ax=ax, s=10, color=sns_green)

ax.set_ylabel('Modulus (GPa)')
ax.legend(['Storage modulus', 'Loss modulus', 'Tan(delta)'])
fig.tight_layout()

fig.savefig(f'{DATA_PATH}/all_data_{common_name}.png', dpi=300)

plt.show()

## 4.1. Looking At The 95% Confidence Interval

In [ ]:
temp = interested_in.copy()
temp.append('source')
err_df = pd.DataFrame(columns=interested_in)
av_temperature = np.zeros(min_length)

# Getting the average temperatures
for df in dfs:
    df = df.iloc[:min_length, :].copy()
    av_temperature = av_temperature + df.loc[:, 'Temperature'].to_numpy().astype(np.float16)
av_temperature = (av_temperature / len(dfs)).round(5)

# concatenating the data
for i, df in enumerate(dfs):
    df = df.iloc[:min_length, :].copy()
    df.loc[:, 'Temperature'] = av_temperature
    df.loc[:, 'source'] = np.zeros(min_length) + i
    err_df = pd.concat([err_df, df], axis=0)

In [ ]:
fig, ax = plt.subplots(dpi=150)
ax.set_yscale('log')

# TODO: since we have only 3-4 replications, is it statistically significance to use confidence interval? `errorbar=("ci", 95)`
# TODO: if we can use ci, should I do Grand-mean centering?

sns.lineplot(data=err_df, x='Temperature', y="Storage modulus", ax=ax, errorbar="se", color=sns_red)
sns.lineplot(data=err_df, x='Temperature', y="Loss modulus", ax=ax, errorbar="se", color=sns_blue)
sns.lineplot(data=err_df, x='Temperature', y="Tan(delta)", ax=ax, errorbar="se", color=sns_green)

ax.set_ylabel('Modulus (GPa)')
ax.legend(['Storage modulus', '', 'Loss modulus', '', 'Tan(delta)', ''])
fig.tight_layout()

# 95%_confidence_interval
fig.savefig(f'{DATA_PATH}/standard error_{common_name}.png', dpi=300)

plt.show()

## 4.2. Smoothing the data and calculating the gradient
Applying a simple moving average to smooth the data

### 4.2.1. Explanation of the code
- `w = 25` sets the width of the moving average window to 25 data points.
- `smooth_y = np.convolve(y1, np.ones(w), 'valid') / w` calculates the moving average of the y1 data by convolving it with a window of ones of width w, and then dividing the result by `w`. The **"valid"** argument ensures that the output has the same length as the input.
- `smooth_x = x[w//2 : -w//2+1]` creates a new array containing the `x` values that correspond to the smoothed data. The slicing operation here removes the first and last `w//2` values of `x` to match the length of `smooth_y`.

The `convolve()` function performs a linear convolution, which is a mathematical operation that computes the integral of the product of two functions as one of them is reversed and shifted over the other. **In the context of signal processing, convolution is a way to combine two signals, and is often used for tasks such as filtering and smoothing.**

The `w // 2` is the **integer division** of w by 2, which gives the number of points to remove from the beginning and end of the x array to match the length of the smoothed data.
For example, if w is 25, `w // 2` is 12. Then, `-w // 2` is -13.

In [ ]:
# Setting the window to anything less than 3 will not smooth the data
w = len(x) // 3
if w > 2:
    smooth_y = np.convolve(y1, np.ones(w), 'valid') / w
    smooth_x = x[(w // 2): (-w // 2) + 1]
else:
    smooth_y = y1
    smooth_x = x

# We need a dataframe to use the query function and filter out the data. Otherwise, we would have to use masking and boolean indexing.
grad = pd.DataFrame([smooth_x, smooth_y]).transpose()
grad.columns = ['temperature', 'gradient']
threshold = 0.5
start_point = min(grad.query(f'gradient > {threshold}')['temperature'])
# end_point = max(grad.query(f'gradient > {threshold}')['temperature'])     # TODO: find a way to find the end point
end_point = max(x)

fig, ax = plt.subplots(dpi=125)
sns.lineplot(x=grad['temperature'],
             y=grad['gradient'],
             ax=ax,
             color=sns_blue)
ax.axvline(x=start_point, color=sns_purple, linestyle='--')
# ax.axvline(x=end_point, color='red', linestyle='--')

print(f"start point: {start_point}")
print(f"end point  : {end_point}")

fig.savefig(f'{DATA_PATH}/gradient_{common_name}.png', dpi=300)

plt.show()

## 4.3. Plotting the data

In [ ]:
# change the colors
plt.style.use('ggplot')
# change the size
# plt.style.use('seaborn-poster')
plt.style.use('seaborn-paper')
# change background
plt.style.use('seaborn-whitegrid')

fig, ax = plt.subplots(dpi=150)

ax.set_yscale('log')

st = ax.scatter(x, y1, color=sns_red, marker='o', s=10, alpha=0.7);
ls = ax.scatter(x, y2, color=sns_blue, marker='o', facecolor='none', s=10)
tan = ax.scatter(x, y3, color=sns_green, marker='^', s=10, alpha=0.7)

ax.legend(['Storage modulus', 'Loss modulus', 'Tan(delta)'])

#ax.set_xlim(10, 85);
#ax.set_ylim(0.01 , 1000);

#ax.axvline(x=start_point, color=sns_purple, linestyle='--')
#ax.axvline(x=end_point, color=sns_purple, linestyle='--')

fig.savefig(f'{DATA_PATH}/av_values_{common_name}.png', dpi=300)

plt.show()

# 5. Curve fitting Storage M.

$$
f(x) = \frac{L}{1 + exp(-k * (x - x_0))} + b
$$

- **L** is responsible for scaling the output range from [0,1] to [0,L]
- **b** adds bias to the output and changes its range from [0,L] to [b,L+b]
- **k** is responsible for scaling the input, which remains in (-inf,inf)
- **x0** is the point in the middle of the Sigmoid, i.e. the point where Sigmoid should originally output the value 1/2
if $x=x_0$, we get $$\frac{1}{1+exp(0)} = 0.5$$

In [ ]:
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k * (x - x0))) + b
    return (y)

filtered_df = av_df[(av_df['Temperature'] > start_point)].copy()
x_filtered = filtered_df.loc[:, 'Temperature'].to_numpy().astype(np.float16)
y1_filtered = filtered_df.loc[:, 'Storage modulus'].to_numpy().astype(np.float16)

y1_log = np.log10(y1_filtered)

p0 = [max(y1_filtered), np.median(x_filtered), 1, min(y1_filtered)]

popt, pcov = curve_fit(sigmoid, x_filtered, y1_log, p0, method='lm')

## 5.1. Looking at the fitted curve

In [ ]:
# change the colors
plt.style.use('ggplot')
plt.style.use('seaborn-paper')
plt.style.use('seaborn-whitegrid')

L_opt ,x0_opt, k_opt, b_opt = popt

x_model = np.linspace(min(x_filtered), max(x_filtered), 100)
y_model = sigmoid(x_model, L_opt, x0_opt, k_opt, b_opt)

fig, ax = plt.subplots(dpi=150)

ax.scatter(x_filtered, y1_log, color=sns_blue, marker='o', s=20)
ax.plot(x_model, y_model, color=sns_red)

plt.show()

## 5.2. Looking at covariance matrix
The diagonals provide the variance of the parameter estimate.
One standard deviation errors on the parameters:

In [ ]:
perr = np.sqrt(np.diag(pcov))
labels = ['L', 'x0', 'k', 'b']
sns.barplot(x=labels, y=perr);

The result is 4x4 heat map. The main diagonal is related to the error (from top left to bottom right) for L, x0, k, b.

In [ ]:
fig, ax = plt.subplots(figsize=[6, 6], dpi=100)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1, alpha=1)
ax.grid(which='major', alpha=0)

im = ax.imshow(error_data:=np.log(np.abs(pcov)), cmap=sns.cubehelix_palette(as_cmap=True, reverse=True))
# plt.cm.inferno)

# Major ticks
ax.set_yticks(np.arange(0, 4, 1))
ax.set_xticks(np.arange(0, 4, 1))

# Labels for major ticks
ax.set_yticklabels(labels, fontsize=12)
ax.set_xticklabels(labels, fontsize=12)

# setting minor ticks for grid
ax.set_xticks(np.arange(-.5, 3.5, 1), minor=True, alpha=0)
ax.set_yticks(np.arange(-.5, 3.5, 1), minor=True, alpha=0)

# Creating a more advanced color map
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", "4%", pad="2%")
cbar = plt.colorbar(im, cax=cax)
cbar.ax.set_ylabel(ylabel="Error", rotation=90, fontsize=12)
cbar.ax.set_yticklabels(labels=np.arange(round(error_data.min(), 1), round(error_data.max(), 1), 0.5), fontsize=12)
cbar.minorticks_on()

plt.show()
